In [78]:
from gensim.models import Word2Vec
import gensim
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
# from readingcorp import FUNCTION

In [70]:
from nltk.corpus import brown

In [71]:
sentences = []

with open("harry_potter.txt") as f:
    for line in f:
        sentences.append(word_tokenize(line))

In [105]:
old_sentences = sentences[0:10000]
new_sentences = sentences[10000:20000]
# Load the dictionary from 
# dict = FUNCTION()
# old_sentences = dict["year1"]
# new_sentences = dict["year2"]

In [106]:
model_1 = gensim.models.Word2Vec(old_sentences, window=5)
model_2 = gensim.models.Word2Vec(new_sentences, window=5)

In [107]:
common_words = set(model_1.wv.key_to_index.keys()).intersection(set(model_2.wv.key_to_index.keys()))

In [108]:
similarities = {}
for word in common_words:
    similarities[word] = cosine_similarity(model_1.wv[word].reshape(1,-1),model_2.wv[word].reshape(1,-1))[0][0]

In [124]:
sorted_by_similarity = sorted(similarities.items(), key = lambda x : x[1])

In [127]:
sorted_by_similarity[:30]

[('the', 0.67875874),
 ('of', 0.755246),
 ('and', 0.76004946),
 ('his', 0.7695195),
 ('had', 0.7901448),
 ('fell', 0.7909205),
 ('was', 0.79477),
 ('into', 0.79941356),
 ('a', 0.7995287),
 ('said', 0.8022046),
 ('down', 0.80239725),
 ('eyes', 0.8042687),
 ('over', 0.80637836),
 ('in', 0.8091757),
 ('on', 0.8100218),
 ('as', 0.81065124),
 ('first', 0.81098294),
 ('them', 0.81329215),
 ('terrible', 0.8146811),
 ('their', 0.8151612),
 ('How', 0.8155184),
 ('looked', 0.816066),
 ('hands', 0.81608737),
 ('out', 0.816157),
 ('day', 0.8173875),
 (':', 0.8185295),
 ('pain', 0.8190478),
 ('once', 0.82128215),
 ('full', 0.8215705),
 ('around', 0.8222778)]